# Título do notebook

## TL;DR
Fazer uma descrição de 3 linhas do objetivo do notebook e conclusões presentes na análise.

## Objetivo

Descrever com clareza qual o objetivo desse notebook.

## Dados

Descrever a base de dados utilizada.

## Modelos (opcional)

Em caso de treinamento ou comparação de modelos, descrever os modelos considerados

## Conclusões
Listar **todas** as conclusões obtidas no notebook:
1. conclusão 1
2. conclusão 2
3. etc.


## Observações importantes:
1. Sempre organize o conteúdo utilizando células markdown e seções. Também utilize células de texto para escrever sua análise das figuras que forem geradas.
2. Lembre-se que essa primeira célula deve ter informação suficiente para entender o que foi feito e quais conclusões você chegou, sem ter que ler o notebook inteiro. Isso agiliza muito a busca por informações ao longo do projeto. Além disso, seu eu do futuro (ou novas pessoas que entrem no projeto) vai agradecer se o seu notebook estiver bem escrito e completo.

# Aqui é a primeira seção

In [1]:
import os
import pandas as pd
import numpy as np

# Caminho da pasta base
base_folder = '../../data/data_gabriele'

# Inicializa um dicionário para armazenar os dados
data_dict = {}

# Percorre todos os diretórios de voluntários
for voluntario in sorted(os.listdir(base_folder)):
    voluntario_path = os.path.join(base_folder, voluntario)
    if os.path.isdir(voluntario_path):  # Garante que é uma pasta

        # Percorre os arquivos CSV dentro da pasta do voluntário
        for filename in sorted(os.listdir(voluntario_path)):
            if filename.endswith('.csv'):
                file_path = os.path.join(voluntario_path, filename)

                # Cria nome da coluna: ex: Vol1_003
                col_name = f'{voluntario}_{filename.replace(".csv", "")}'

                # Lê o CSV
                df = pd.read_csv(file_path, header=None)

                # Garante que existe a coluna 22 (índice 21)
                if df.shape[1] > 21:
                    data_dict[col_name] = df.iloc[:, 21].values

# Descobre o maior comprimento de sinal
max_len = max(len(col) for col in data_dict.values())

# Cria a coluna de tempo com passo de 0.0334 segundos
timestamps = np.arange(0, max_len * 0.0334, 0.0334)

# Cria o DataFrame final
final_df = pd.DataFrame({'timestamp': timestamps})

# Adiciona os sinais
for col_name, signal in data_dict.items():
    # Preenche com NaN se o sinal for menor que o maior comprimento
    padded_signal = np.pad(signal, (0, max_len - len(signal)), constant_values=np.nan)
    final_df[col_name] = padded_signal

# Exibe uma prévia do DataFrame
print(final_df.head())
print(f"Shape final: {final_df.shape}")


   timestamp  voluntario1_003  voluntario1_004  voluntario1_005  \
0     0.0000         153.5869         140.6963         137.6439   
1     0.0334         153.2465         140.7868         137.6439   
2     0.0668         153.2465         140.7868         137.6439   
3     0.1002         153.0471         140.7868         137.6439   
4     0.1336         153.0471         140.8961         137.6439   

   voluntario1_006  voluntario1_007  voluntario1_008  voluntario1_009  \
0         140.5624         146.8580         150.8253         157.6682   
1         140.5624         146.8547         150.8253         157.6682   
2         140.5624         146.8547         150.8253         157.6682   
3         140.5624         146.8547         150.8253         157.6682   
4         140.5624         146.7946         150.8253         157.6682   

   voluntario1_010  voluntario2_001  ...  voluntario5_001  voluntario5_002  \
0         163.6071         192.2271  ...         228.3185         237.3994   
1 

In [3]:
final_df.to_csv('../../data/GabrieleDB.csv', index=False)

In [ ]:
import os
import pandas as pd
import numpy as np
import re # Importa o módulo de expressões regulares

# Apontar para a pasta principal que contém as subpastas dos voluntários
main_folder = './data_gabriele' # <-- Altere aqui para o caminho correto da sua pasta

# Inicializa um dicionário para armazenar os dados
data_dict = {}

# Índices das colunas a serem extraídas
cols_to_extract = [18, 19, 20, 21, 22, 23]

# Usa os.walk() para percorrer a pasta principal e todas as suas subpastas
print(f"Iniciando varredura no diretório: {main_folder}")
for root, dirs, files in os.walk(main_folder):
    # Ordena os arquivos para garantir uma sequência lógica (001, 002, ...)
    for filename in sorted(files):
        if filename.endswith('.csv'):
            
            # --- LÓGICA DE NOMENCLATURA MODIFICADA PARA O FORMATO 'vol1ex1' ---
            
            # Pega o nome da pasta (ex: "voluntario1")
            volunteer_folder_name = os.path.basename(root)
            # Pega o número do exercício do nome do arquivo (ex: "003")
            exercise_number_str = os.path.splitext(filename)[0]
            
            # Usa regex para extrair apenas o número do nome da pasta do voluntário
            vol_num_match = re.search(r'\d+$', volunteer_folder_name)
            
            # Continua apenas se ambos os nomes de pasta e arquivo estiverem no formato esperado
            if vol_num_match:
                vol_num = vol_num_match.group() # Extrai o número '1' de 'voluntario1'
                
                try:
                    # Converte o número do exercício para inteiro para remover zeros à esquerda (ex: '003' -> 3)
                    exercise_num = int(exercise_number_str)
                    
                    # Constrói o nome base no formato desejado: 'vol1' + 'ex' + '1' -> "vol1ex1"
                    clean_name = f"vol{vol_num}ex{exercise_num}"
                    
                    # --- Fim da lógica de nomenclatura ---

                    file_path = os.path.join(root, filename)
                    
                    # Lê o CSV
                    df = pd.read_csv(file_path, header=None)
                    
                    # Lógica para percorrer e nomear x, y, z para quadril e joelho
                    if df.shape[1] > max(cols_to_extract):
                        for idx, col_idx in enumerate(cols_to_extract):
                            if idx < 3:
                                axis = f'q{"xyz"[idx]}' # Quadril
                            else:
                                axis = f'j{"xyz"[idx - 3]}' # Joelho
                            
                            # Nome final da coluna, ex: "vol1ex1_qx"
                            col_name = f"{clean_name}_{axis}"
                            data_dict[col_name] = df.iloc[:, col_idx].values
                    else:
                        print(f"Aviso: Arquivo '{file_path}' tem menos colunas que o esperado e será ignorado.")

                except ValueError:
                    print(f"Aviso: O nome do arquivo '{filename}' não é um número válido e será ignorado.")
            else:
                # Isso acontece se a pasta não terminar com um número (ex: a própria pasta 'data_gabriele')
                pass

# --- PARTE FINAL PARA MONTAR O DATAFRAME (SEM ALTERAÇÃO) ---
print("\nTodos os arquivos foram lidos. Montando o DataFrame final...")

if not data_dict:
    print("Nenhum dado foi extraído. Verifique o caminho da pasta e a estrutura dos arquivos.")
else:
    max_len = max(len(col) for col in data_dict.values())
    timestamps = np.arange(0, max_len * 0.0334, 0.0334)[:max_len]
    final_df = pd.DataFrame({'timestamp': timestamps})

    for col_name, signal in data_dict.items():
        padded_signal = np.pad(signal.astype(float), (0, max_len - len(signal)), mode='constant', constant_values=np.nan)
        final_df[col_name] = padded_signal

    print("\nPrévia do DataFrame gerado com a nova nomenclatura:")
    print(final_df.head())
    print(f"\nDimensões do DataFrame: {final_df.shape}")

    # Ordena as colunas na ordem desejada (timestamp, vol1ex1..., vol1ex2..., etc.)
    # Extrai vol + número e ex + número para ordenar corretamente
    def sort_key(col_name):
        if col_name == 'timestamp':
            return (0, 0, '') # Garante que timestamp venha primeiro
        match = re.match(r'vol(\d+)ex(\d+)_([qj][xyz])', col_name)
        if match:
            vol_num = int(match.group(1))
            ex_num = int(match.group(2))
            axis = match.group(3)
            return (vol_num, ex_num, axis)
        return (999, 999, col_name) # Fallback para colunas inesperadas

    sorted_columns = sorted(final_df.columns, key=sort_key)
    final_df = final_df[sorted_columns]

    print("\nPrévia do DataFrame com colunas ordenadas:")
    print(final_df.head())

    output_filename = 'dados_extraidos_formato_final.csv'
    final_df.to_csv(output_filename, index=False)
    print(f"\nDataFrame salvo com sucesso como '{output_filename}'")

Iniciando varredura no diretório: ./data_gabriele

Todos os arquivos foram lidos. Montando o DataFrame final...

Prévia do DataFrame gerado:
   timestamp  voluntario1_ex003_qx  voluntario1_ex003_qy  \
0     0.0000              -3.31250              87.54699   
1     0.0334               5.59375              94.80544   
2     0.0668               5.59375              94.80544   
3     0.1002               5.59375              94.99655   
4     0.1336               5.59375              94.99655   

   voluntario1_ex003_qz  voluntario1_ex003_jx  voluntario1_ex003_jy  \
0              200.8948              153.5869               77.6674   
1              218.8205              153.2465               77.6674   
2              218.8205              153.2465               77.6674   
3              219.9629              153.0471               77.6674   
4              219.9629              153.0471               77.6674   

   voluntario1_ex003_jz  voluntario1_ex004_qx  voluntario1_ex004_qy  \


C:\Users\Gabri\AppData\Local\Temp\ipykernel_11348\4172037962.py:77: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  final_df[col_name] = padded_signal
C:\Users\Gabri\AppData\Local\Temp\ipykernel_11348\4172037962.py:77: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  final_df[col_name] = padded_signal
C:\Users\Gabri\AppData\Local\Temp\ipykernel_11348\4172037962.py:77: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all 


DataFrame salvo com sucesso como 'dados_extraidos_Gabriele.csv'


# Aqui é a segunda seção

In [29]:
...

Ellipsis

# Conclusões

Aqui você deve listar todas as conclusões obtidas ao longo do notebook.
Pode copiar todo o conteúdo dessa seção lá na parte de conclusões do cabeçalho.